In [1]:
import numpy as np
import pickle
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import StratifiedKFold
from data_generation import m_0, g_0, get_data

In [2]:
# Load tuned hyperparameters of ElasticNet
with open('opt_params_eln.pkl', 'rb') as pickle_file:
    opt_params_eln = pickle.load(pickle_file)
with open('opt_params_eln_stand.pkl', 'rb') as pickle_file:
    opt_params_eln_stand = pickle.load(pickle_file)

In [3]:
opt_params_eln_stand

{250: {0: {'g0': {'alpha': 0.06083721732445604, 'l1_ratio': 1.0},
   'g1': {'alpha': 0.03462496718147671, 'l1_ratio': 1.0},
   'm': {'C': 0.08858667904100823, 'l1_ratio': 0.3}},
  1: {'g0': {'alpha': 0.025371416870198553, 'l1_ratio': 1.0},
   'g1': {'alpha': 0.022938377439128806, 'l1_ratio': 1.0},
   'm': {'C': 0.23357214690901212, 'l1_ratio': 0}},
  2: {'g0': {'alpha': 0.03969712398533498, 'l1_ratio': 1.0},
   'g1': {'alpha': 0.044934743064839466, 'l1_ratio': 1.0},
   'm': {'C': 0.08858667904100823, 'l1_ratio': 0.1}}},
 500: {0: {'g0': {'alpha': 0.03724454070124745, 'l1_ratio': 1.0},
   'g1': {'alpha': 0.057628075870434996, 'l1_ratio': 1.0},
   'm': {'C': 0.23357214690901212, 'l1_ratio': 1}},
  1: {'g0': {'alpha': 0.016088644772676085, 'l1_ratio': 1.0},
   'g1': {'alpha': 0.026293256333984646, 'l1_ratio': 1.0},
   'm': {'C': 0.08858667904100823, 'l1_ratio': 0.1}},
  2: {'g0': {'alpha': 0.03255289579014974, 'l1_ratio': 1.0},
   'g1': {'alpha': 0.03625280169739805, 'l1_ratio': 1.0},
   

In [3]:
# Get ElasticNet models from hyperparameters
def get_models(eln_params_dict):
    model_g = []
    for d in [0, 1]:
        model = ElasticNet(max_iter=10000)
        model.set_params(**eln_params_dict[f'g{d}'])
        model_g.append(model)
    model_m = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=50000, random_state=42)
    model_m.set_params(**eln_params_dict['m'])
    return model_g, model_m

In [4]:
mse, mse_stand = {}, {}

for N in opt_params_eln.keys():
    rng = np.random.default_rng(seed=23)
    mse_N, mse_stand_N = {}, {}
    
    for j in range(3):
        y_data, d_data, x_data = get_data(N, rng)
        poly_features = PolynomialFeatures(degree=2, include_bias=False)
        x_data_quad = poly_features.fit_transform(x_data)
        scaler = StandardScaler()
        x_data_quad_stand = scaler.fit_transform(x_data_quad)

        model_g, model_m = get_models(opt_params_eln[N][j])
        model_g_stand, model_m_stand = get_models(opt_params_eln_stand[N][j])
        mse_N_j, mse_stand_N_j = {'g0':[], 'g1':[], 'm':[]}, {'g0':[], 'g1':[], 'm':[]}

        skf = StratifiedKFold(n_splits=5, shuffle=False)
        for (train_indices, eval_indices) in skf.split(X=x_data, y=d_data):
            y_train, d_train, x_train, x_train_quad, x_train_quad_stand = y_data[train_indices], d_data[train_indices], x_data[train_indices], x_data_quad[train_indices], x_data_quad_stand[train_indices]
            y_eval, d_eval, x_eval, x_eval_quad, x_eval_quad_stand = y_data[eval_indices], d_data[eval_indices], x_data[eval_indices], x_data_quad[eval_indices], x_data_quad_stand[eval_indices]

            # Estimate outcome regression functions g_0(d)
            for d in [0, 1]:
                model_g[d].fit(X=x_train_quad[d_train==d], y=y_train[d_train==d])
                mse_N_j[f'g{d}'].append(mean_squared_error(g_0(d, x_eval), model_g[d].predict(x_eval_quad)))
                model_g_stand[d].fit(X=x_train_quad_stand[d_train==d], y=y_train[d_train==d])
                mse_stand_N_j[f'g{d}'].append(mean_squared_error(g_0(d, x_eval), model_g_stand[d].predict(x_eval_quad_stand)))
                
            # Estimate propensity score m_0
            model_m.fit(X=x_train_quad, y=d_train)
            mse_N_j['m'].append((mean_squared_error(m_0(x_eval), model_m.predict_proba(x_eval_quad)[:,1]), mean_absolute_error(m_0(x_eval), model_m.predict_proba(x_eval_quad)[:,1])))
            model_m_stand.fit(X=x_train_quad_stand, y=d_train)
            mse_stand_N_j['m'].append((mean_squared_error(m_0(x_eval), model_m_stand.predict_proba(x_eval_quad_stand)[:,1]), mean_absolute_error(m_0(x_eval), model_m_stand.predict_proba(x_eval_quad_stand)[:,1])))

        mse_N[j] = mse_N_j
        mse_stand_N[j] = mse_stand_N_j

    mse[N] = mse_N
    mse_stand[N] = mse_stand_N

In [5]:
with open('mse.pkl', 'wb') as pickle_file:
    pickle.dump(mse, pickle_file)
with open('mse_stand.pkl', 'wb') as pickle_file:
    pickle.dump(mse_stand, pickle_file)

In [7]:
for N in opt_params_eln.keys():
    for j in range(3):
        for name in ['g0', 'g1', 'm']:
            print(N, j, name, 
                  np.round(np.mean(mse[N][j][name], axis=0), 6), 
                  np.round(np.mean(mse_stand[N][j][name], axis=0), 6))

250 0 g0 0.789994 0.693069
250 0 g1 0.638813 0.523678
250 0 m [0.025789 0.124724] [0.025457 0.124566]
250 1 g0 0.995499 0.791502
250 1 g1 0.896472 0.97954
250 1 m [0.045774 0.15349 ] [0.041571 0.148672]
250 2 g0 0.699959 0.503817
250 2 g1 0.885154 0.794347
250 2 m [0.026581 0.119684] [0.026645 0.122718]
500 0 g0 0.577841 0.489899
500 0 g1 0.462872 0.383998
500 0 m [0.014295 0.088911] [0.015584 0.092768]
500 1 g0 0.526686 0.424751
500 1 g1 0.447857 0.333216
500 1 m [0.018864 0.102457] [0.016706 0.093027]
500 2 g0 0.426806 0.394677
500 2 g1 0.559694 0.505032
500 2 m [0.01537  0.092089] [0.015859 0.092264]
1000 0 g0 0.319913 0.267559
1000 0 g1 0.32812 0.328177
1000 0 m [0.007219 0.061698] [0.007719 0.063465]
1000 1 g0 0.274829 0.265229
1000 1 g1 0.305812 0.265117
1000 1 m [0.006965 0.058896] [0.007183 0.060708]
1000 2 g0 0.268174 0.233851
1000 2 g1 0.321088 0.317638
1000 2 m [0.009932 0.073198] [0.009574 0.07644 ]
2000 0 g0 0.201058 0.178244
2000 0 g1 0.228407 0.231519
2000 0 m [0.00404  